In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131322 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Build neural network 
import numpy as np
import cv2

IMAGE_SIZE = 224
IMAGE_DATA = '/content/gdrive/My Drive/app/face_boss/data'

images = []
labels = []

import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

model = keras.applications.mobilenet.MobileNet(classes=2, weights=None)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
import os 
import random
from shutil import copyfile

def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

# Generate dataset

for path, subdirs, files in os.walk(IMAGE_DATA):
    for name in files:
        
        img_path = os.path.join(path, name)
        #print(img_path)
        images.append(prepare_image(img_path))
        labels.append(path.split('/')[-1])


In [0]:
images = np.array(images)

In [0]:
images = np.squeeze(images, axis=1)

In [0]:

mapped_labels = list(map(lambda x: 1 if x == 'boss' else 0, labels))

from keras.utils import np_utils

y_data = np_utils.to_categorical(mapped_labels)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, y_data, test_size=0.2)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [0]:
model.compile(loss='categorical_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint

model_file = "/content/gdrive/My Drive/app/face_boss/boss_model/model_mobilenet.hdf5"

checkpoint = ModelCheckpoint(model_file, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
model.fit(x=X_train, y=y_train, batch_size=16, epochs=100, verbose=1, validation_data=(X_test, y_test), callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 364 samples, validate on 92 samples
Epoch 1/100
364/364 [==============================] - 22s 61ms/step - loss: 0.4759 - acc: 0.7665 - val_loss: 1.9132 - val_acc: 0.6957

Epoch 00001: val_loss improved from inf to 1.91319, saving model to /content/gdrive/My Drive/app/face_boss/boss_model/model_mobilenet.hdf5
Epoch 2/100
364/364 [==============================] - 5s 13ms/step - loss: 0.4671 - acc: 0.7582 - val_loss: 2.1830 - val_acc: 0.6957

Epoch 00002: val_loss did not improve from 1.91319
Epoch 3/100
364/364 [==============================] - 5s 13ms/step - loss: 0.4133 - acc: 0.7775 - val_loss: 3.5603 - val_acc: 0.6957

Epoch 00003: val_loss did not improve from 1.91319
Epoch 4/100
364/364 [==============================] - 5s 13ms/step - loss: 0.3811 - acc: 0.7802 - val_loss: 4.9055 - val_acc: 0.6957

Epoch 00004: val_loss did not improve from 1.91

In [0]:
model.save('/content/gdrive/My Drive/app/face_boss/boss_model/model.h5')

In [9]:
from keras.models import load_model
model = load_model('/content/gdrive/My Drive/app/face_boss/boss_model/model.h5')
x = prepare_image('/content/gdrive/My Drive/app/face_boss/test.jpg')
#print(x)

print(np.argmax(model.predict(x)[0]))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
1


In [10]:
x = prepare_image('/content/gdrive/My Drive/app/face_boss/Abdullah_Gul_0006.jpg')

print(np.argmax(model.predict(x)[0]))


0


In [11]:
x = prepare_image('/content/gdrive/My Drive/app/face_boss/test.jpg')
#print(x)

print(np.argmax(model.predict(x)[0]))

1
